# Seasonal Forecasts (concatenating all years and creating final datasets)

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from tqdm import tqdm


## File Creation

In [ ]:
def file_creation(all, variable, name):
    
    temp = all.where(variable, drop=True)

    comp = dict(zlib=True, complevel=9)
    encoding = {var: comp for var in temp.data_vars}

    temp.to_netcdf(path='/data/ibougoudis/MOAD/files/' + name + '.nc', mode='a', encoding=encoding)
    temp.close()


## Main

In [ ]:
# Files loading
path = '/data/ibougoudis/MOAD/files/'
files = os.listdir(path)[0:2]

# Collecting them all
all = []
for i in files:
    all.append(xr.open_dataset(path+i))

all = xr.concat((all[:]), dim='time_counter')

# Time indexes
jan_mar_i = ((all.time_counter.dt.month == 1) & (all.time_counter.dt.day > 15) | ((all.time_counter.dt.month == 2) & (all.time_counter.dt.day < 29)) | (all.time_counter.dt.month == 3))
jan_apr_i = ((all.time_counter.dt.month == 1) | ((all.time_counter.dt.month == 2) & (all.time_counter.dt.day < 29)) | (all.time_counter.dt.month == 3) | (all.time_counter.dt.month == 4))
apr_jun_i = ((all.time_counter.dt.month == 4) & (all.time_counter.dt.day > 15) | (all.time_counter.dt.month == 5) | (all.time_counter.dt.month == 6))

# File creation
file_creation(all, jan_mar_i, 'jan_mar')
file_creation(all, jan_apr_i, 'jan_apr')
file_creation(all, apr_jun_i, 'apr_jun')
